In [14]:
from image_utils import *
from skimage.transform import rescale,resize
from scipy import misc
from numpy import newaxis
import numpy as np

In [15]:
#Dataset from SPACENET for Vegas, Paris, Khartum and Shanghai have resolution of 30 cm per pixel for pansharpen whereas
# GHANA has 50 cm but for Rio it is also 50 cm

ratio_ghana=3.0/5.0




In [16]:
def align_resolution_with_ghana(image):
    '''
    Aligns resolution of images from SPACENET (Vegas, Paris, Khartum and Shanghai) which are not the same than Ghana dataset
    :image ndarray with dimension (number of channels x width x height)
    return ndarray with dimension ( width x height x number of channels)
    '''
    image=np.transpose(image,(1,2,0))
    return rescale(image,ratio_ghana)[newaxis,:,:,:]

In [17]:
def prepare_ms_hr(ms_lr,size_hr):
    '''
    Prepares the upsampled MS image 
    :ms_lr input image to upsample
    :size_hr to upsample the Low Resolution MS image to the dimension of the High Resolution panchromatic image
    '''
    ms_lr=np.transpose(ms_lr,(1,2,0))
    return resize(ms_lr, size_hr)[newaxis,:,:,:]


In [18]:
def prepare_panchro(pansharp_rgb_path):
    return misc.imread(pansharp_rgb_path,'L')[newaxis,:,:,newaxis]
    

In [22]:
if __name__ == '__main__':
    
    path='/scratch/SPACENET_DATA/BUILDING_DATASET/'
    
    path_out='/scratch/SPACENET_DATA_PROCESSED/'
    path_raw=path_out+'RAW_IMAGES/'
    if not os.path.exists(path_raw):
                os.makedirs(path_raw)
    
    ## build RAW patches folders
    for filedir in os.listdir(path):
        if not os.path.exists(path_raw+filedir+'/PANCHRO/'):
                os.makedirs(path_raw+filedir+'/PANCHRO/')
        if not os.path.exists(path_raw+filedir+'/PANSHARP/'):
                os.makedirs(path_raw+filedir+'/PANSHARP/')
        if not os.path.exists(path_raw+filedir+'/GROUNDTRUTH/'):
                os.makedirs(path_raw+filedir+'/GROUNDTRUTH/')
        
    
    
    for filedir in os.listdir(path):

        if filedir.startswith('AOI_1_RIO'):
            _3_bands_folder=path+'AOI_1_RIO/processedBuildingLabels/3band/'
            _8_bands_folder=path+'AOI_1_RIO/processedBuildingLabels/8band/'
            geojson_folder=path+'AOI_1_RIO/processedBuildingLabels/vectordata/geojson/'
            file=filedir.split('AOI_')[1]
            print('Preparing %s'%file)
            
            panchro_size=[]
            panchro_files=[]
            for filename in  sorted(os.listdir(_3_bands_folder))[:2]:
                file=filename.split('RIO_')[1]
                panchro_files.append(_3_bands_folder+filename)

                print('Preparing Panchro %s'%file)
                panchro=prepare_panchro(_3_bands_folder+filename)
                panchro_size.append(panchro.shape[1:3])
                print('Panchro Shape [%d,%d,%d,%d]'%panchro.shape)

                write_data_h5(path_raw+'AOI_1_RIO/PANCHRO/panchro_1_RIO_'+file.split('.tif')[0]+'.h5',panchro)
                
            panchro_size=np.asarray(panchro_size)  
            panchro_files=np.asarray(panchro_files)
            count=0
            for filename in  sorted(os.listdir(_8_bands_folder))[:2]:
                file=filename.split('RIO_')[1]
                print('Preparing Pansharp %s'%file)
                pansharp=read_images(_8_bands_folder+filename)
                pansharp=prepare_ms_hr(pansharp,panchro_size[count])
                print('Pansharp Shape [%d,%d,%d,%d]'%pansharp.shape)
                write_data_h5(path_raw+'AOI_1_RIO/PANSHARP/pansharp_1_RIO_'+file.split('.tif')[0]+'.h5',pansharp)
                count+=1     
            count=0
            for filename in  sorted(os.listdir(geojson_folder))[:2]:
                file=filename.split('RIO_')[1]
                createRasterFromGeoJson(geojson_folder+filename, panchro_files[count],path_raw+'AOI_1_RIO/GROUNDTRUTH/groundtruth_1_RIO_'+file.split('.geojson')[0]+'.png')
                labels=read_labels(path_raw+'AOI_1_RIO/GROUNDTRUTH/groundtruth_1_RIO_'+file.split('.geojson')[0]+'.png')
                os.remove(path_raw+'AOI_1_RIO/GROUNDTRUTH/groundtruth_1_RIO_'+file.split('.geojson')[0]+'.png') 
                print('Groundtruth shape [%d,%d,%d,%d]'%labels[newaxis,:,:,newaxis].shape)
                write_data_h5(path_raw+'AOI_1_RIO/GROUNDTRUTH/groundtruth_1_RIO_'+file.split('.geojson')[0]+'.h5',labels[newaxis,:,:,newaxis])
                count+=1
            
        else:

            panchro_folder=path+filedir+'/'+filedir+'_Train/PAN/'
            pansharp_folder=path+filedir+'/'+filedir+'_Train/MUL-PanSharpen/'
            geojson_folder=path+filedir+'/'+filedir+'_Train/geojson/buildings/'
            file=filedir.split('AOI_')[1]
            print('Preparing %s'%file)
            panchro_files=[]
            for filename in  sorted(os.listdir(panchro_folder))[:2]:
                file=filename.split('AOI_')[1]
                panchro_files.append(panchro_folder+filename)
                print('Preparing Panchro %s'%file)
                panchro=read_images(panchro_folder+filename)
                panchro=align_resolution_with_ghana(panchro[newaxis,:,:])
                print('Panchro Shape [%d,%d,%d,%d]'%panchro.shape)
                write_data_h5(path_raw+filedir+'/PANCHRO/'+'panchro_'+file.split('.tif')[0]+'.h5',panchro)
           
            panchro_files=np.asarray(panchro_files)
            count=0          
            for filename in  sorted(os.listdir(pansharp_folder))[:2]:
                file=filename.split('AOI_')[1]
                print('Preparing Pansharp %s'%file)
                pansharp=read_images(pansharp_folder+filename)
                pansharp=align_resolution_with_ghana(pansharp)
                print('Pansharp Shape [%d,%d,%d,%d]'%pansharp.shape)
                write_data_h5(path_raw+filedir+'/PANSHARP/'+'pansharp_'+file.split('.tif')[0]+'.h5',pansharp)
            for filename in  sorted(os.listdir(geojson_folder))[:2]:
                file=filename.split('AOI_')[1]
                createRasterFromGeoJson(geojson_folder+filename, panchro_files[count],path_raw+filedir+'/GROUNDTRUTH/'+'groundtruth_'+file.split('.geojson')[0]+'.png')
                labels=read_labels(path_raw+filedir+'/GROUNDTRUTH/'+'groundtruth_'+file.split('.geojson')[0]+'.png')
                os.remove(path_raw+filedir+'/GROUNDTRUTH/'+'groundtruth_'+file.split('.geojson')[0]+'.png')
                labels=align_resolution_with_ghana(labels[newaxis,:,:])
                print('Groundtruth shape [%d,%d,%d,%d]'%labels.shape)
                write_data_h5(path_raw+filedir+'/GROUNDTRUTH/'+'groundtruth_'+file.split('.geojson')[0]+'.h5',labels)
                count+=1
            
        

Preparing 1_RIO
Preparing Panchro img1.tif
Panchro Shape [1,406,438,1]
File/scratch/SPACENET_DATA_PROCESSED/RAW_IMAGES/AOI_1_RIO/PANCHRO/panchro_1_RIO_img1.h5 created
Preparing Panchro img10.tif
Panchro Shape [1,406,438,1]
File/scratch/SPACENET_DATA_PROCESSED/RAW_IMAGES/AOI_1_RIO/PANCHRO/panchro_1_RIO_img10.h5 created
Preparing Pansharp img1.tif
Pansharp Shape [1,406,438,8]
File/scratch/SPACENET_DATA_PROCESSED/RAW_IMAGES/AOI_1_RIO/PANSHARP/pansharp_1_RIO_img1.h5 created
Preparing Pansharp img10.tif
Pansharp Shape [1,406,438,8]
File/scratch/SPACENET_DATA_PROCESSED/RAW_IMAGES/AOI_1_RIO/PANSHARP/pansharp_1_RIO_img10.h5 created
Groundtruth shape [1,406,438,1]
File/scratch/SPACENET_DATA_PROCESSED/RAW_IMAGES/AOI_1_RIO/GROUNDTRUTH/groundtruth_1_RIO_img1.h5 created
Groundtruth shape [1,406,438,1]
File/scratch/SPACENET_DATA_PROCESSED/RAW_IMAGES/AOI_1_RIO/GROUNDTRUTH/groundtruth_1_RIO_img10.h5 created
Preparing 2_Vegas
Preparing Panchro 2_Vegas_img1.tif
Panchro Shape [1,390,390,1]
File/scratch/S

/home/s161362/.conda/envs/env_dhi/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  
/home/s161362/.conda/envs/env_dhi/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


Pansharp Shape [1,390,390,8]
File/scratch/SPACENET_DATA_PROCESSED/RAW_IMAGES/AOI_2_Vegas/PANSHARP/pansharp_2_Vegas_img1.h5 created
Preparing Pansharp 2_Vegas_img10.tif
Pansharp Shape [1,390,390,8]
File/scratch/SPACENET_DATA_PROCESSED/RAW_IMAGES/AOI_2_Vegas/PANSHARP/pansharp_2_Vegas_img10.h5 created
Groundtruth shape [1,390,390,1]
File/scratch/SPACENET_DATA_PROCESSED/RAW_IMAGES/AOI_2_Vegas/GROUNDTRUTH/groundtruth_2_Vegas_img1.h5 created
Groundtruth shape [1,390,390,1]
File/scratch/SPACENET_DATA_PROCESSED/RAW_IMAGES/AOI_2_Vegas/GROUNDTRUTH/groundtruth_2_Vegas_img10.h5 created
Preparing 3_Paris
Preparing Panchro 3_Paris_img10.tif
Panchro Shape [1,390,390,1]
File/scratch/SPACENET_DATA_PROCESSED/RAW_IMAGES/AOI_3_Paris/PANCHRO/panchro_3_Paris_img10.h5 created
Preparing Panchro 3_Paris_img100.tif
Panchro Shape [1,390,390,1]
File/scratch/SPACENET_DATA_PROCESSED/RAW_IMAGES/AOI_3_Paris/PANCHRO/panchro_3_Paris_img100.h5 created
Preparing Pansharp 3_Paris_img10.tif
Pansharp Shape [1,390,390,8]
Fil